# Battle of Neighbourhoods - Week 1

This notebook consist of two parts.

### 1 Introduction/Business Problem

    1.a Discussion of the business problem and the audience who would be interested in this project.

### 2 Data Section

    2.a What data is used?
    2.b Importing Libraries
    2.c Credentials and Core location
    2.d Search for hotel & restaurant within 1 KM
    2.e Location of Hotels
   
   
## 1. Introduction/Business Problem
Discussion of the business problem and the audience who would be interested in this project.

### TOPIC: Opening a new Hotel / Restaurant in Kanyakumari, India 
### About the place - Kanyakumari, India

Kanyakumari,a beautiful __tourist spot__ in southern India.Kanyakumari is a coastal town in the state of Tamil Nadu on India's southern tip. Jutting into the Laccadive Sea, the town was known as 'Cape Comorin'. A popular tourist destination in India, it is famous for its unique ocean sunrise, sunset and moonrise  over the ocean, the 133 feet Thiruvalluvar Statue and Vivekananda Rock Memorial off the coast, and as a pilgrimage centre. Lying at the tip of peninsular India, it is the confluence of the Arabian sea, the Bay of Bengal and the Indian Ocean. Other historic sights that you can visit in Kanyakumari include the Padmanabhapuram Palace, the Vattakottai Fort, and the Gandhi Memorial.

### Opening of Hotel/Restaurant
Regarding opening any business problem, It would be better to open a hotel/restaurant near beach side.As it is a famous tourist spot,there is already lots of attention towards it. There will be many competitors in terms of hotel and restauramt.But keeping them in mind,the new hotel should be opened in a place where more people are attracted and comfortable for a stay and a good meal. IT will attract foreign and local peoples attention towards the new hotel. And the flavour new retaurant recipe with Italian, American, typical South & North Indian foods to grab their taste.

### Business Problem

The objective of this capstone project is to analyse and select the ___suitable locations___ in the city of Kanyakumari,India for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.  Using data science methodology and machine learning techniques like clustering, this project aims to provide solutions to answer the business question.

### Expected / Interested Visitors
80% local and 20% foreign peoples visit kanyakumari once in a year.Some people stay for couple of days or more. Also they find some place for hangout or a good meal. Their main focus might be belonging to stay somewhere near to reach venues. Apart from these set of people, students and working professionals are common visitors here. So we may need to fascinate them all.

## 2.Data section

### 2.a What data is used?

* We will be completely working on __Foursquare data__ to explore and try to locate our new hotel where more venues like church, temples, beach, museums, memorials that are present nearby.




__To solve the problem, we will need the following data:__

* List of neighbourhoods in Kanyakumari.

* Latitude and longitude coordinates of those neighbourhoods. 

* Venue data related to Hotel/Resaurant. This will help us determine the most suitable place to open a Hotel/Resaurant.

### How will we be solving using this data?

We will looking for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of kanyakumari.

Just a heads up on how many hotels are distributed now around kanyakumari.We will perform some EDA(Exploratory Data Analysis) on hotels & restaurants present in the tourist spot.On furthur notebook we will use Foursquare data to determine other venues as well.

## 2. b Importing Libraries

In [1]:
# Import libraries
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library
import numpy as np # data in a vectorized manner manipulation

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [3]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans # clustering algorithm

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("*********   Loaded library     ***********")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

## 2.c Credentials and Core location

In [21]:
CLIENT_ID = 'RRSUUY4OMR0ZGWYDKT31GGWXSKEACXUX0PCL30XGYHVVC1LS'
CLIENT_SECRET = 'AW3ISA52EYVITQ3OR12ITTD1CQFB4O2ICATTQ1D2BGB5YRLX'
VERSION = '20200811'
LIMIT = 150

address = "Kanyakumari, Tamil Nadu"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #8.079252 # location.latitude 
longitude =location.longitude #77.5499338 # location.longitude # 

kan='Kanyakumari location : {},{}'.format(latitude,longitude)
print(kan)

Kanyakumari location : 8.079252,77.5499338


## 2.d  Search for hotel & restaurant within 1 KM

In [22]:
#Quering for hotel & restaurant using foursquare API

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 2000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

__Send the GET Request of hotel & restaurants and examine the results__

In [23]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

__Get relevant part of JSON and transform it into a pandas dataframe__

In [24]:
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at Kanyakumari".format(dataframe.shape[0]))

There are 40 restaurants and hotels at Kanyakumari


__Define information of interest and filter dataframe__

In [25]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Hotel SeaView,Hotel,371,8.082333,77.551217,4cd42e2c558a370412d0c5a6
1,Hotel Samudra,Motel,201,8.080642,77.551102,4d3c66e414aa8cfa4641a95e
2,Hotel Sarvana,Hotel,179,8.080581,77.550861,4db052544df03036e8b9d615
3,Hotel Annapoorna,Indian Restaurant,206,8.080879,77.550835,50ed3a39e4b0ac50005eb3f1
4,Hotel Trisea,South Indian Restaurant,269,8.081475,77.548969,5030a43ee4b021f7a59c7456


## 2.e Location of Hotels

In [26]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Kanyakumari

# add a red circle marker to represent the core location of kanyakumari
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kanyakumari',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

# More to come in next notebook- "Battle of Neighbourhoods - Final"